In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('../data/raw/april_movie_scores_v2.json')

In [3]:
df.head()

,Title,Critic_score,Audience_score,Details,Rating,URL,Poster_URL
0,Cop-Out,,,"1967, Drama/Crime, 1h 35m",,https://www.rottentomatoes.com/m/cop_out-1967,https://resizing.flixster.com/HOKDkfbwskhg7LCE...
1,Coven of Sisters,83,84,"2020, Drama, 1h 30m",,https://www.rottentomatoes.com/m/coven_of_sisters,https://resizing.flixster.com/zhHs_fy9ekb7au5s...
2,Friends With Benefits,68,65,"2011, Romance, 1h 49m",R,https://www.rottentomatoes.com/m/friends_with_...,https://resizing.flixster.com/s8BnY0xP6GCJwyPv...
3,Insidious: The Last Key,33,50,"2018, Horror/Mystery and thriller, 1h 43m",PG-13,https://www.rottentomatoes.com/m/insidious_the...,https://resizing.flixster.com/dI2Ao2aOyPNKmNIg...
4,Irul,0,,"2021, Mystery and thriller, 1h 31m",,https://www.rottentomatoes.com/m/irul,https://resizing.flixster.com/0LeWXambtQWf7BHS...


In [4]:
# View how many rows are missing critic scores
df[df.Critic_score == ""].count()

Title             178
Critic_score      178
Audience_score    178
Details           178
Rating            178
URL               178
Poster_URL        178
dtype: int64

In [5]:
# View how many rows are missing audience scores
df[df.Audience_score == ""].count()

Title             69
Critic_score      69
Audience_score    69
Details           69
Rating            69
URL               69
Poster_URL        69
dtype: int64

In [6]:
# Determine count of missing values across dataframe
df.replace("", np.NaN, inplace=True)
df.isnull().sum()

Title               0
Critic_score      178
Audience_score     69
Details             0
Rating            193
URL                 0
Poster_URL          0
dtype: int64

In [7]:
# Replace missing Critic_score, Audience_score, and Rating values with NR
df.Critic_score.replace(np.NaN, "NR", inplace=True)
df.Audience_score.replace(np.NaN, "NR", inplace=True)
df.Rating.replace(np.NaN, "NR", inplace=True)

In [8]:
df.isnull().sum()

Title             0
Critic_score      0
Audience_score    0
Details           0
Rating            0
URL               0
Poster_URL        0
dtype: int64

In [9]:
# View first value of Details column
df.Details[0]

'1967, Drama/Crime, 1h 35m'

In [10]:
# Extract values from Details column to create new columns
df['Year'] = df.Details.str.split(",").str[0].str.strip()
df['Title_category'] = df.Details.str.split(",").str[1]
df['Title_category'] = df.Title_category.str.strip()
df['Runtime'] = df.Details.str.split(",").str[2].str.strip()

# Drop Details from df
df.drop('Details', axis=1, inplace=True)
df.head(2)

,Title,Critic_score,Audience_score,Rating,URL,Poster_URL,Year,Title_category,Runtime
0,Cop-Out,NR,NR,NR,https://www.rottentomatoes.com/m/cop_out-1967,https://resizing.flixster.com/HOKDkfbwskhg7LCE...,1967,Drama/Crime,1h 35m
1,Coven of Sisters,83,84,NR,https://www.rottentomatoes.com/m/coven_of_sisters,https://resizing.flixster.com/zhHs_fy9ekb7au5s...,2020,Drama,1h 30m


In [11]:
# Reorder dataframe to match SQL table
columns = ['Title', 'Year', 'Critic_score', 'Audience_score', 'Title_category', 'Rating', 'Runtime', 'URL', 'Poster_URL']
df = df[columns]
df.head(2)

,Title,Year,Critic_score,Audience_score,Title_category,Rating,Runtime,URL,Poster_URL
0,Cop-Out,1967,NR,NR,Drama/Crime,NR,1h 35m,https://www.rottentomatoes.com/m/cop_out-1967,https://resizing.flixster.com/HOKDkfbwskhg7LCE...
1,Coven of Sisters,2020,83,84,Drama,NR,1h 30m,https://www.rottentomatoes.com/m/coven_of_sisters,https://resizing.flixster.com/zhHs_fy9ekb7au5s...


In [12]:
# Check count of duplicates based on Title, Year, Critic_score, Audience_score
df.duplicated(subset=['Title', 'Year', 'Critic_score', 'Audience_score']).value_counts()

False    339
True       1
dtype: int64

In [13]:
# View duplicated values based on Title, Year, Critic_score, Audience_score
df[df.duplicated(subset=['Title', 'Year', 'Critic_score', 'Audience_score'], keep=False)]

,Title,Year,Critic_score,Audience_score,Title_category,Rating,Runtime,URL,Poster_URL
270,Slayers: The Motion Picture,1995,NR,74,Animation/Fantasy,NR,1h 15m,https://www.rottentomatoes.com/m/slayers_great,/assets/pizza-pie/images/poster_default.c8c896...
272,Slayers: The Motion Picture,1995,NR,74,Animation/Fantasy,NR,1h 15m,https://www.rottentomatoes.com/m/slayers_the_m...,/assets/pizza-pie/images/poster_default.c8c896...


In [14]:
# Drop those duplicated values, keeping the last one
df = df.drop_duplicates(subset=['Title', 'Year', 'Critic_score', 'Audience_score'], keep='last')

In [15]:
# Omit Poster_URL values with .gif as these are bad URLs
df.loc[df.Poster_URL.str.contains('.gif'), 'Poster_URL'] = ""